In [ ]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [ ]:
url_inicial='https://revistas.chapingo.mx/horticultura/index.php?section=articles&subsec=issues'
url_root= 'https://revistas.chapingo.mx/horticultura/index.php?section=articles&subsec=issues'
r = requests.get(url_inicial)

In [ ]:
soup = BeautifulSoup(r.text, 'html.parser')

In [ ]:
box = soup.find('div', id='accordionExample')
volumen=box.findAll('a')
vol = [x.get('href')for x in volumen]
vol=[urljoin(url_root,i) for i in vol]
vol

In [ ]:
vol2 =[]
for i in vol: 
    url_inicial1=i 
    r1 = requests.get(url_inicial1)
    soup1 = BeautifulSoup(r1.text, 'html.parser')
    box1 = soup1.findAll('a', class_='doi')
    vol1 = [x.get('href')for x in box1]
    vol2+=vol1

In [ ]:
vol2

In [ ]:
def get_url_items(sopa, url): 
    box1 = soup1.findAll('a', class_='doi')
    vol1 = [x.get('href')for x in box1]
    #vol=[urljoin(url_root,i) for i in vol]
    return vol2

In [ ]:

links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links) 
    next_a=s_p.select('li.next > a')
    if not next_a or not next_a[0].get('href'):
        break
    url_inicial = urljoin(url_inicial, next_a[0].get('href'))

In [ ]:
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

In [ ]:
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

In [ ]:
def scraper_book(url):
    content_book={}
    r=requests.get(url)
    tr='Revista Chapingo Serie Horticultura'
    a='VI. Biotecnología y Ciencias Agropecuarias'
    te='Horticultura'
    s_item=BeautifulSoup(r.text,'html.parser')
    tituloR=tr
    if tituloR:
        content_book['Titulo Revista']=tituloR
    else :
        content_book['Titulo Revista']=None
    area=a
    if area:
        content_book['Area']=area
    else :
        content_book['Area']=None
    tematica=te
    if tematica:
        content_book['Tematica']=tematica
    else :
        content_book['Tematica']=None
    #titulo del libro
    try:
        titulo=s_item.find('h1', class_='titulos').get_text(strip=True)
        content_book['Titulo Articulo']=titulo
    except AttributeError:
        content_book['Titulo Articulo']=None
    #obtener descripción
    try:
        des=s_item.find('div', class_='panel-bodyy').get_text(strip=True)
        content_book['Resumen']=des
    except AttributeError:
        content_book['Resumen']=None
    try:
        de=s_item.find('h2', class_='subtext')
        content_book['Abstract']=de.find_next_sibling('p').get_text(strip=True)
    except AttributeError:
        content_book['Abstract']=None
    try:
        urlA=s_item.find('h1', class_='titulos')
        content_book['Link Articulo']=urlA.find_next_sibling('a').get('href')
    except AttributeError:
        content_book['Link Articulo']=None 
        #obtener link PDF
    try:
        ancla_link=s_item.find('li', class_='download').find('a').get('href')
        content_book['Link PDF']=urljoin(url_root,ancla_link)
    except AttributeError:
        content_book['Link PDF']=None
    return content_book

In [ ]:
list_scraper=list_scraper[0:98]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

In [ ]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

In [ ]:
df_catalogo.to_csv('Revista046.csv', index=False)